In [ ]:
!apt-get install sqlite3

In [2]:
import sqlite3

In [3]:
import os

In [6]:
if os.path.exists('PortfolioProject.db'): #Substitute with your DB name
    os.remove('PortfolioProject.db')
    print('Database deleted.')
else:
    print('Database does not exist.')


Database deleted.


In [8]:
conn = sqlite3.connect('PortfolioProject.db') #Substitute with your DB name

IN THE ABOVE STEP WE MADE A CONNECTION TO THE DATABASE AND ACTIVATED IT FOR DATA QUERIES.

In [ ]:
#Substitute with your Table name
conn.execute('''CREATE TABLE mytable 
               (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)''')

In [ ]:
#conn.execute("INSERT INTO mytable (name, age) VALUES ('Sai', 23), ('Akhil', 29), ('Chintu', 24)")
#conn.execute("DELETE FROM mytable WHERE Age = 25 ")

In [ ]:
cursor = conn.execute("SELECT * FROM mytable")
for row in cursor:
    print(row)


In [ ]:
cursor = conn.execute("SELECT name FROM mytable")
for row in cursor:
    print(row)


In [ ]:
conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('PortfolioProject.db') #Substitute with your DB name

CREATED THE PORTFOLIO PROJECT DATABASE

In [10]:
import pandas as pd
#import sqlite3

# Connect to the database
conn = sqlite3.connect('PortfolioProject.db')

# Read the data from the Excel file into a pandas DataFrame
df = pd.read_excel('CovidDeaths.xlsx')

# Insert the data from the DataFrame into the database table
df.to_sql('CovidDeaths', conn, if_exists='append', index=False)




85171

In [11]:

# Connect to the database
conn = sqlite3.connect('PortfolioProject.db')

# Read the data from the Excel file into a pandas DataFrame
df = pd.read_excel('CovidVaccinations.xlsx')

# Insert the data from the DataFrame into the database table
df.to_sql('CovidVaccinations', conn, if_exists='append', index=False)




85171

EXTRACTED THE DATA FROM EXCEL USING PANDAS AND LOADED TO DB TABLE FROM DF

In [12]:
conn.commit()

THING TO EXECUTE YOUR SQL QUERY USING SQLITE AND COLAB

In [ ]:
# cursor = conn.execute("")
# for row in cursor:
#     print (row)

In [ ]:
cursor = conn.execute("SELECT * FROM CovidVaccinations LIMIT 5")
for row in cursor:
    print(row)


In [ ]:
cursor = conn.execute("SELECT * FROM CovidDeaths LIMIT 5")
for row in cursor:
    print(row)


In [ ]:
#This is the Required Data we are going to use
cursor = conn.execute("SELECT Location, Date, Total_cases, new_cases, total_deaths, population FROM CovidDeaths ORDER BY 1,2 LIMIT 5")
for row in cursor:
    print(row)


In [ ]:
#Looking Total cases and Total Deaths
#Shows of the Likelihood of Dying if you are in India at that time 
cursor = conn.execute("SELECT Location, Date, Total_cases, total_deaths, (total_deaths/total_cases)*100 as DeathPercentage FROM CovidDeaths WHERE location LIKE '%india%' ORDER BY 1,2 LIMIT 4,49")
for row in cursor:
    print(row)

In [ ]:
#Looking Total cases vs Population
#It shows the percentage of the population who got covid
cursor = conn.execute("SELECT Location, Date, Total_cases, population, (total_cases/population)*100 as InfectedPercentage FROM CovidDeaths WHERE location LIKE '%india%' ORDER BY 1,2 LIMIT 4,49")
for row in cursor:
    print(row)

In [ ]:
#Looking to countries at Highest Infection Rate
cursor = conn.execute("SELECT location, population, MAX(total_cases) AS HighestInfectionCount, MAX(total_cases/population)*100 as percentagepopulationinfected FROM CovidDeaths GROUP  BY location, population ORDER BY percentagepopulationinfected DESC")
for row in cursor:
    print (row)

In [ ]:
#Looking into countries with High Mortality
#Looking to countries at Highest Infection Rate
cursor = conn.execute("SELECT location, MAX(cast(total_deaths AS INT)) AS TotalDeathCount FROM CovidDeaths WHERE continent IS NOT NULL GROUP  BY location ORDER BY TotalDeathCount DESC LIMIT 5")
for row in cursor:
    print (row)

In [ ]:
cursor = conn.execute("SELECT * FROM CovidDeaths WHERE continent IS NOT NULL LIMIT 5")
for row in cursor:
    print(row)


In [ ]:
#BREAKING THIS BY CONTINENT
cursor = conn.execute("SELECT continent, MAX(cast(total_deaths AS INT)) AS TotalDeathCount FROM CovidDeaths WHERE continent IS  NOT NULL GROUP  BY continent ORDER BY TotalDeathCount DESC LIMIT 10")
for row in cursor:
    print (row)

In [ ]:
#GLOBAL NUMBERS
cursor = conn.execute("SELECT Date, SUM(new_cases) as Total_New_cases,SUM(CAST(new_deaths AS INT)) as Total_New_deaths, SUM(CAST(new_deaths AS INT))/SUM(new_cases)FROM CovidDeaths WHERE continent IS NOT NULL GROUP BY DATE ORDER BY 1,2 LIMIT 4,49")
for row in cursor:
    print(row)

In [ ]:
#TOTAL POLULATION VS VACCINATED
cursor = conn.execute("SELECT dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations, SUM(CAST(vac.new_vaccinations AS INTEGER)) OVER (PARTITION BY dea.location ORDER BY dea.location,dea.date) as RollingPeopleVaccinated FROM CovidDeaths dea JOIN CovidVaccinations vac ON dea.location=vac.location AND vac.date=dea.date WHERE dea.continent IS NOT NULL ORDER BY dea.location, dea.date LIMIT 9000, 9100")
for row in cursor:
    print (row)

In [ ]:
#CTE 

PopvsVac = """
WITH PopvsVac AS (
    SELECT 
        dea.continent, 
        dea.location, 
        dea.date, 
        dea.population, 
        vac.new_vaccinations, 
        SUM(CAST(vac.new_vaccinations AS INTEGER)) OVER (PARTITION BY dea.location ORDER BY dea.location, dea.date) as RollingPeopleVaccinated 
    FROM 
        CovidDeaths dea 
        JOIN CovidVaccinations vac ON dea.location=vac.location AND vac.date=dea.date 
    WHERE 
        dea.continent IS NOT NULL 
    LIMIT 
        9000, 9100
)

SELECT *, (RollingPeopleVaccinated/Population) FROM PopvsVac;
"""

# Execute the query and print the results
cursor = conn.execute(PopvsVac)
for row in cursor:
    print(row)

In [ ]:
# drop the table if it already exists
conn.execute("DROP TABLE IF EXISTS temp_PercentPopulationVaccinated")

# create the table
conn.execute("CREATE TABLE temp_PercentPopulationVaccinated (Continent nvarchar(255), Location nvarchar(255), Date datetime, Population numeric, New_vaccinations numeric, RollingPeopleVaccinated numeric)")

# insert data into the table
query = ("""
Insert into temp_PercentPopulationVaccinated
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
From PortfolioProject..CovidDeaths dea
Join PortfolioProject..CovidVaccinations vac
	On dea.location = vac.location
	and dea.date = vac.date
--where dea.continent is not null 
--order by 2,3
""")
cursor = conn.execute(query)


result = conn.execute("SELECT *, (RollingPeopleVaccinated/Population)*100 FROM temp_PercentPopulationVaccinated")
for row in result:
    print(row)



In [ ]:
%%bigquery
CREATE VIEW PercentPopulationVaccinated AS
SELECT dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations,
SUM(CONVERT(int,vac.new_vaccinations)) OVER (PARTITION BY dea.Location ORDER BY dea.location, dea.Date) as RollingPeopleVaccinated
FROM `your-project-name.PortfolioProject.CovidDeaths` dea
JOIN `your-project-name.PortfolioProject.CovidVaccinations` vac
ON dea.location = vac.location AND dea.date = vac.date
WHERE dea.continent IS NOT NULL
